# Modelling:

#### 1. What is bias variance trade-off?

* The Bias-Variance trade-off is the central problem in supervised machine learning. Ideally, one wants to choose a model that accurately captures the regularities in its training data, but also generilizes well to unseen data. Unfortunately, these goals are contradictory, and often impossible to do both. 
* __Bias__ represents the error as a result of misaligned assumptions in the learning algorithm that do not represent the true relationship between predictors and the response variable. 
* __Variance__ represents the error from sensitivity to fluctuations in the training set. High variance can cause an algorithm to model the noise in the training data, rather than intended outputs. 
* __Discussion__ Models with low-bias are usually more complex (ex: higher-order regression polynomials), enabling them to represent the training set from accurately. However, they may also represent the noise in the trianing set, making their predictions less accurate. On the other hand, models with high-bias are simple (ex: linear regression polynomials), but may produce lower variance prediction when applied beyond the training set. 
* __Sources:__ [wiki](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff)


#### 2. Derive bias variance trade-off?


* __Derivation of Variance:__  
$Var(X) = E[(X-E[X])^2]$  
$Var(X) = E[(X^2 + E[X]^2 - 2 * X E[X])]$  
$Var(X) = E[X^2] + E[X]^2 - 2 * E[X] * E[X]$  
$Var(X) = E[X^2] - E[X]^2$  

* __Variance in $y$__  
$ Var(y) = E[ (y - E[y])^2]$  
$ Var(y) = E[ (E[y] + e - E[y])^2]$  
$ Var(y) = E[e^2]$  
$ Var(y) = Var(e) + E[e]^2$; But: $E[e] = 0$  
$ Var(y) = Var(e)$  

* __Sum of Squares Error:__  
$SS = E[(y - \hat{y})^2]$  
$SS = E[(y^2 + \hat{y}^2 - 2 * y * \hat{y}]$  
$SS = E[y^2] + E[\hat{y}^2] - 2 E(y*\hat{y})$  
$SS = Var(y) + E[y]^2  + Var(\hat{y}) + E[\hat{y}]^2  - 2 E(y*\hat{y})$
$SS = Var(e) + Var(\hat{y}) + \left( E[y] - E[\hat{y}]\right)^2$  
$SS = irreducible\space error + Variance + Bias^2$



3. Linear regression derivation
4. Linear regression derivation based on matrix
5. Write CART algorithm
6. Write Bagged Tree algorithm
7. Write Random Forest algorithm
8. Write Gradient boosting algorithm
9. Derive NN - backpropagation
10. What is SVN?
11. What is ARIMA formula structure? What are the steps to take in trianing a good time series model? What are alternatives to ARIMA?
12. How would you tell whether a new feature X is beneficial or not  
13. Create a model to predict a response variable
14. What could be issues if distribution of test data is significantly different from training data?
15. How to make model more robust to outliers?
16. MSE vs. MAD why choose one? Differences in model that uses one over the other. 
17. How to evaluate a binary classifier?
18. How to deal with imbalanced binary response variable?
19. What models can be used to predict a binary response variable? What are are the differences between these?
20. What is regularization and where might it be helpful? What is an example of regularization in a model?
21. Why might it be better to include fewer features compared to many?
22. Given training data on tweets and their tweets, how would you predict the no. of re-tweets of a given tweet after 7 days after only observing 2 days worth of data?
23. How would you construct a feed to show relevant content for a site that involved user interactions with items?
24. How would you deisgn the people you may know feature on Linkedin or FB?
25. How would you predict who someone may want to send a Snapchat or Gmat to?
26. How would you suggest to a franchise where to open a new store?
27. In a search engine, query auto complete solution
28. Given a database of all previous alumni donations to your university, how would you predict which recent alumni are more likely to donate?
29. You're Uber and you want to design a heatmap to recommend to drivers where to wait for passenger. How would you approach this?
30. How would you build a model to predict a March Madness bracket?
31. You want to run a regression to predict the probability of a flight delay, but there are flights with delays up to 12 hours that are really messing up your model. How will you address this?
32. Derive MLE estimation from likelihood function?
33. Program a Naive Bayes algorithm
34. Program a k-NN algorithm (a) iterative and (b) vectorized
35. What are differences between generative and discriminative models? Examples.
36. Derive likelihood function for BG NBD
37. Typical loss functions used: 
    a. Least Squared Error
    b. Logistic Loss
    c. Hinge Loss
    d. Cross-entropy
38. What is the cost function in gradient descent?
39. What is the general form of gradient descent? What are the typical parameters to control gradient descent?
40. What is Newton's algorithm? What is the Newton-Rahpson method?
41. Derive linear regression co-efficients using Normal equation?
42. What is the mathematical form of Least Mean Square Algorithm?
43. What is locally weighted regression? 
44. What is the sigmoid function
45. What is the general form of logistic regression
46. What is the genral form of softmax regression?
47. What are generalized regression models? How can the Bernauli, Gaussian, Poisson, Geomertric distributions be used within GLM?
48. What are the assumptions in GLM?
49. What is SVM? How does it work?
50. How does Gaussian Discriminant analysis work?
51. Learning Theory: What is Union Bound?
52. Learning Theory: What is Hoeffding inequality?
53. Learning Theory: What is Training Error?
54. Learning Theory: What is Probably Approximately Correct (PAC)?
55. Learning Theory: What is Shattering?
56. Learning Theory: What is Upper Bound Theorem?
57. Learning Theory: What is VC dimension?
58. Learning Theory: What is Vapnik Theorem?
59. What is EM algorithm? How is it used in discovering LAten variables?
60. What is k-means clustering? How does the algorithm work?
61. How to optimize k-means? how to find optimal k?
62. What are the assumptions with k-means algorithm?
63. What is k-protype algorithm? When is it typically used?
64. What is Hierarchical clustering? What are the different linkage functions that can be used?
65. What are the metrics that can be used to assess clusters? Solihouette, Calinski-Harabaz?
66. What is PCA? How does it work? Code it up
67. What is ICA? How does it work?
68. What is NN? What is the general form?
69. What are the typical activation functions used? Describe Sigmoid, Tanh, ReLU, Leaku ReLU?
70. How does the backpropagation algorithm work?
    a. Cross-entropy loss
    b. Learning Rate
    c. Backpropagation
    d. Updating weights
    e. Dropout
71. What is CNN?
    a. Convolutional Layer Requirement
    b. Batch normalization
72. What is RNN?
    a. Types of gates? Input, Forget, Gate, Output gate
    b. LSTM
73. Re-inforcement Learning and Control
    a. What is policy
    b. What is markov decision process
    c. What is value function?
    d. What are bellman equation
    e. Value iteration algorithm
    f. MLE
    g. Q-learning
74. Classification evaluation metrics:
    a. Accuracy, Precision, Recal (Sensitivity), Specificity, F1 Score
    b. ROC (TPR aka Recall aka Sensitivity, FPR aka 1- specificity)
    c. AUC



 

#### 75. Regression Metrics:
* **Total Sum of Squares (TSS):** Measures the variation in the observed data $$ \sum_{i=0}^{m} (y_i - \bar{y})^2$$  
* __Residual Sum of Squares (RSS):__ Measures the variation in the modelling errors $$ \sum_{i=0}^{m} (y_i - \hat{y_i})^2$$  
* __Explained Sum of Squares (ESS):__  Measures variation in the modelled values $$ \sum_{i=0}^{m} (\hat{y_i} - \bar{y})^2$$ 
* $R^2$ = $$ (1- \frac{RSS}{TSS})$$  
* Model performance: Mallow's Cp, AIC, BIC, Adjusted R^2


#### 76. Regularization:
* __Definition:__ Regularization is a technique used to reduce the complexity of the model, the objective is to increase bias and avoid overfitting the training sample. Regularization is usually done through addition of regularization term to the cost function of the machine learning model. 
* __LASSO aka L1 Regularization:__ Lasso shriks co-efficients towards 0. When the $\lambda$ is sufficiently large, the lasso method is likely to end up  shrinking some of the coefficients to 0. If there is a group of highly correlated variables, Lasso tends to select one from the group and ignore the rest. $$J(\theta) = \sum_{i = 1}^{m}{\left( y_i - \theta_0 - \sum_{j=1}^{n} \theta_j x_{i\space j}\right) ^2} + \lambda \sum_{j=1}^{n}{\mid \theta_j \mid}$$
* __Ridge aka L2 Regulization:__ Ridge regularization shriks co-efficient to 0. But, due to the nature of the penalty term, ridge penalization always yields models that have all the $n$ predictors. $$J(\theta) = \sum_{i = 1}^{m}{\left( y_i - \theta_0 - \sum_{j=1}^{n} \theta_j x_{i\space j}\right) ^2} + \lambda \sum_{j=1}^{n}{ \theta_j}^2$$
* __Elastic Net:__ L1 regularization is conservative with highly-correlated variables. Elastic net combines the cost function of both L1 and L2 to allow for more flexbility when compared to L1. 
$$J(\theta) = \sum_{i = 1}^{m}{\left( y_i - \theta_0 - \sum_{j=1}^{n} \theta_j x_{i\space j}\right) ^2} + \lambda_1 \sum_{j=1}^{n}{\mid \theta_j \mid} + \lambda_2 \sum_{j=1}^{n}{ \theta_j}^2$$


#### 77. Diagnostics:
* Discovering overfitting, underfitting through training and cv error
